## Fine-tuning Trainer API

In [4]:
## define a TrainingArguments class to contain all the hyperparameters the Trainer will use for training/evaluation

from transformers import TrainingArguments

# help(TrainingArguments)

args = TrainingArguments(
    output_dir='misc/files/training_out/',
)

In [5]:
## define the model

from transformers import AutoModelForSequenceClassification

classifier = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
